[![Kaggle link](https://img.shields.io/badge/изначальный%20датасет-Открыть-orange?style=social&logo=kaggle)](https://www.kaggle.com/datasets/leomartinelli/bullying-in-schools)


In [68]:
df_csv_path = '/Users/blacksnow/bmstu/ММО/Лабораторные работы/datasets/Bullying_2018_raw.csv'

## §1. Зависимости

In [69]:
import pandas as pd
import pandas.io.sql as psql
from tabulate import tabulate
import numpy as np
%matplotlib inline 

## §2. Описание изначального датасета

In [70]:
bullying_df = pd.read_csv(df_csv_path) 

In [79]:
import psycopg2

conn = psycopg2.connect (
    host='localhost',
    database='bullying',
    user='blacksnow',
    password='password'
)

In [71]:
bull_desc = [
    ['Номер ответа','-'],
    ['В школе (за последние 12 месяцев)','Категориальный'], # - №2
    ['Вне школы (за последние 12 месяцев)','Категориальный'], # - №3
    ['Кибербуллинг (последние 12 месяцев?)','Категориальный'], # - №3
    ['Возраст респондента','Числовой'], # - №4
    ['Пол респондента','Категориальный'], # - №5
    ['Количество физических атак','Числовой'], # - №6
    ['Количество драк','Числовой'], # # - №7
    ['Ощущение одинокости','Категориальный'], # # - №8
    ['Количество близких друзей','Числовой'], # - №9
    ['Количество дней пропусков школы без разрешения','Числовой '], # - №10
    ['Доброта и отзывчивость окружающих','Категориальный'], # - №11
    ['Частота проблем взаимопонимания с родителями','Категориальный'], # - №12
    ['Ощущение одинокости большую части (частота)','Категориальный'], # # - №13
    ['Пропускали ли вы школу без разрешения (да/нет)','Категориальный'], # - №14
    ['Масса вашего тела в школе была ниже нормы?','Категориальный'], # - №15
    ['Масса вашего тела в школе была выше нормы?','Категориальный'], # - №16
    ['Вы страдали ожирением?','Категориальный'], # - №17
]


In [77]:
select * from bullying;

SyntaxError: invalid syntax (129754482.py, line 1)

## §3. Проверка на пропуски изначального датасета

In [72]:
bullying_df.columns

Index(['record', 'Bullied_on_school_property_in_past_12_months',
       'Bullied_not_on_school_property_in_past_12_months',
       'Cyber_bullied_in_past_12_months', 'Custom_Age', 'Sex',
       'Physically_attacked', 'Physical_fighting', 'Felt_lonely',
       'Close_friends', 'Miss_school_no_permission',
       'Other_students_kind_and_helpful', 'Parents_understand_problems',
       'Most_of_the_time_or_always_felt_lonely',
       'Missed_classes_or_school_without_permission', 'Were_underweight',
       'Were_overweight', 'Were_obese'],
      dtype='object')

In [73]:

df_variants = []

for col in bullying_df.columns:
    print(f"Варианты колонки: {col}: ")
    variants = bullying_df[col].unique()
    table = np.array(variants)
    if table.shape[0]<10:
        table=table.transpose()
        print(tabulate(table))
    print('\n')

Варианты колонки: record: 


Варианты колонки: Bullied_on_school_property_in_past_12_months: 
-  -  -
Y  e  s
N  o

-  -  -


Варианты колонки: Bullied_not_on_school_property_in_past_12_months: 
-  -  -
Y  e  s
N  o

-  -  -


Варианты колонки: Cyber_bullied_in_past_12_months: 
-  -  -

N  o
Y  e  s
-  -  -


Варианты колонки: Custom_Age: 
-  -    -  -  -  -  -    -  -  -    -  -    -  -  -  -  -  -  -
1  3    y  e  a  r  s    o  l  d
1  4    y  e  a  r  s    o  l  d
1  6    y  e  a  r  s    o  l  d
1  2    y  e  a  r  s    o  l  d
1  5    y  e  a  r  s    o  l  d
1  1    y  e  a  r  s    o  l  d    o  r    y  o  u  n  g  e  r
1  7    y  e  a  r  s    o  l  d

1  8    y  e  a  r  s    o  l  d    o  r    o  l  d  e  r
-  -    -  -  -  -  -    -  -  -    -  -    -  -  -  -  -  -  -


Варианты колонки: Sex: 
-  -  -  -  -  -
F  e  m  a  l  e
M  a  l  e

-  -  -  -  -  -


Варианты колонки: Physically_attacked: 
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
0     t  i  m  e  s
1     t  i 

### §3.1 Пропуски

В датасете есть пропущенные данные - для правильного их восприятия заменим все пробелы на nil

Регулярное выражение для Yes/No ^(?:Yes|No)$

In [74]:
bullying_df_clean = bullying_df.copy()

y_or_n_count = bullying_df_clean['Bullied_not_on_school_property_in_past_12_months'].isin(['Yes','No']).count()
total_count = bullying_df_clean['Bullied_not_on_school_property_in_past_12_months'].count()

total_count-y_or_n_count

0

## §4. Статистика по датасету

In [75]:

def stats(data=bullying_df, desc=bull_desc, headers=('Номер','Название','Описание','Кат.','Числ.','Кол-во вариаций')):
    headers=('Номер','dtype','Описание','Тип','Кол-во пропусков','Кол-во вариаций')
    table = np.arange(data.shape[1]*len(headers))
    table = table.reshape(len(headers),data.shape[1])

    table = np.array([
                [x for x in range(1,19)],
                [d for d in data.dtypes],
                [a[0] for a in desc],
                [a[1] for a in desc],
                [data[col].isna().sum() for col in data.columns],
                [data[col].nunique() for col in data.columns]])

    table = table.transpose()

    print(
        tabulate(
            [data.shape],
            headers=('Кол-во строк', 'Кол-во столбцов'),
            tablefmt="github"),
        end='\n\n\n'
        )
    
    print(
        tabulate(
            table,
            headers=headers,
            tablefmt='github'
        ),
        end='\n\n\n'
        )

stats()

|   Кол-во строк |   Кол-во столбцов |
|----------------|-------------------|
|          56981 |                18 |


|   Номер | dtype   | Описание                                       | Тип            |   Кол-во пропусков |   Кол-во вариаций |
|---------|---------|------------------------------------------------|----------------|--------------------|-------------------|
|       1 | int64   | Номер ответа                                   | -              |                  0 |             56981 |
|       2 | object  | В школе (за последние 12 месяцев)              | Категориальный |                  0 |                 3 |
|       3 | object  | Вне школы (за последние 12 месяцев)            | Категориальный |                  0 |                 3 |
|       4 | object  | Кибербуллинг (последние 12 месяцев?)           | Категориальный |                  0 |                 3 |
|       5 | object  | Возраст респондента                            | Числовой       |                  0 

## §4. Нормализация данных

In [76]:
def normalized_source(df=bullying_df):
    return df

stats(data=normalized_source())


|   Кол-во строк |   Кол-во столбцов |
|----------------|-------------------|
|          56981 |                18 |


|   Номер | dtype   | Описание                                       | Тип            |   Кол-во пропусков |   Кол-во вариаций |
|---------|---------|------------------------------------------------|----------------|--------------------|-------------------|
|       1 | int64   | Номер ответа                                   | -              |                  0 |             56981 |
|       2 | object  | В школе (за последние 12 месяцев)              | Категориальный |                  0 |                 3 |
|       3 | object  | Вне школы (за последние 12 месяцев)            | Категориальный |                  0 |                 3 |
|       4 | object  | Кибербуллинг (последние 12 месяцев?)           | Категориальный |                  0 |                 3 |
|       5 | object  | Возраст респондента                            | Числовой       |                  0 

### §4.1 Datenluecken Loeschung